In [14]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, roc_auc_score, balanced_accuracy_score
import lightgbm as lgb
from tabulate import tabulate
import optuna
import shap
import matplotlib.pyplot as plt

csv_files = {
    'Normal': r'C:\Users\prags\Desktop\hackathon\Normal_statistics.csv',
    'Complex_Partial_Seizures': r'C:\Users\prags\Desktop\hackathon\Complex_Partial_Seizures_statistics.csv',
    'Electrographic_Seizures': r'C:\Users\prags\Desktop\hackathon\Electrographic_Seizures_statistics.csv',
    'Video_detected_Seizures_with_no_visual_change_over_EEG': r'C:\Users\prags\Desktop\hackathon\Video_detected_Seizures_with_no_visual_change_over_EEG_statistics.csv',
}

all_features, all_labels = [], []
for label, file_path in csv_files.items():
    data = pd.read_csv(file_path)
    imputer = SimpleImputer(strategy='mean')
    data_imputed = imputer.fit_transform(data)
    all_features.append(pd.DataFrame(data_imputed, columns=data.columns))
    all_labels.extend([label] * len(data_imputed))

X = pd.concat(all_features, ignore_index=True)
y = pd.Series(all_labels)
label_mapping = {label: idx for idx, label in enumerate(y.unique())}
y_encoded = y.map(label_mapping)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

def train_and_evaluate_model(model, X_train, X_val, y_train, y_val, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    y_pred_prob = model.predict_proba(X_val)
    roc_auc = roc_auc_score(y_val, y_pred_prob, multi_class='ovr', average='macro')
    balanced_accuracy = balanced_accuracy_score(y_val, y_pred)
    
    # Safely set num_params to None as no direct equivalent exists
    num_params = None

    return {
        "Model": model_name,
        "ROC-AUC Score": roc_auc,
        "Balanced Accuracy Score": balanced_accuracy,
        "Trainable Parameters": num_params
    }

def objective(trial):
    params = {
        'objective': 'multiclass',
        'num_class': len(label_mapping),
        'metric': 'multi_logloss',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 31, 255),
        'max_depth': trial.suggest_int('max_depth', 5, 50),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-5, 10),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-5, 10),
        'random_state': 42,
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_pred_prob = model.predict_proba(X_val)
    roc_auc = roc_auc_score(y_val, y_pred_prob, multi_class='ovr', average='macro')
    return roc_auc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best parameters found by Optuna: ", study.best_params)

best_params = study.best_params
best_lgb_model = lgb.LGBMClassifier(**best_params)
best_lgb_model.fit(X_train, y_train)

best_model_result = train_and_evaluate_model(best_lgb_model, X_train, X_val, y_train, y_val, "LightGBM Optimized")
print("\nOptimized Model Evaluation:")
print(tabulate([best_model_result], headers="keys", tablefmt="pretty"))

explainer = shap.TreeExplainer(best_lgb_model)
shap_values = explainer.shap_values(X_train)

eeg_channel_columns = X.columns[:10]

top_channels = {}

for class_idx, class_name in enumerate(label_mapping.keys()):
    class_shap_values = shap_values[class_idx]
    feature_importance = np.abs(class_shap_values).mean(axis=0)
    top_indices = feature_importance.argsort()[-3:][::-1]
    top_channel_names = eeg_channel_columns[top_indices].to_list()
    top_channels[class_name] = top_channel_names

class_label_mapping = {
    0: 'Normal',
    1: 'Complex_Partial_Seizures',
    2: 'Electrographic_Seizures',
    3: 'Video_detected_Seizures_with_no_visual_change_over_EEG'
}

table = []
for class_idx, class_name in enumerate(top_channels):
    table.append([class_label_mapping[class_idx]] + top_channels[class_name])

headers = ["Class"] + [f"Top {i+1} EEG Channel" for i in range(3)]
print("\nTop 3 Important EEG Channels for Each Class:")
print(tabulate(table, headers=headers, tablefmt="pretty"))

# Plot SHAP values for all classes
for class_idx, class_name in enumerate(label_mapping.keys()):
    shap.summary_plot(shap_values[class_idx], X_train, feature_names=X.columns, show=False)

# Alternatively, use the first class (normal) for the plot
shap.summary_plot(shap_values[0], X_train, feature_names=X.columns)


[I 2025-01-19 01:11:28,216] A new study created in memory with name: no-name-7efb0834-2106-4aec-a04a-f0b0f1ffdf2c
C:\Users\prags\AppData\Local\Temp\ipykernel_20496\2408327102.py:61: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 0.1),
C:\Users\prags\AppData\Local\Temp\ipykernel_20496\2408327102.py:65: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 1.0),
C:\Users\prags\AppData\Local\Temp\ipykernel_20496\2408327102.py:66: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/rel

[LightGBM] [Warning] min_data_in_leaf is set=77, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=77
[LightGBM] [Warning] feature_fraction is set=0.6463428502251317, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6463428502251317
[LightGBM] [Warning] lambda_l1 is set=1.5471221472771397, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.5471221472771397
[LightGBM] [Warning] lambda_l2 is set=0.00012159665931329487, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00012159665931329487
[LightGBM] [Warning] bagging_fraction is set=0.8397002306462444, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8397002306462444
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] min_data_in_leaf is set=77, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=77
[LightGBM] [Warning] feature_fraction is set=0.6463428502251317, colsamp

[I 2025-01-19 01:11:28,479] Trial 1 finished with value: 0.9974666666666667 and parameters: {'learning_rate': 0.0009618862471166624, 'num_leaves': 74, 'max_depth': 36, 'min_data_in_leaf': 71, 'feature_fraction': 0.6378539054937816, 'bagging_fraction': 0.643507482801458, 'bagging_freq': 1, 'lambda_l1': 0.265654305443842, 'lambda_l2': 0.0002100197464695182}. Best is trial 1 with value: 0.9974666666666667.
C:\Users\prags\AppData\Local\Temp\ipykernel_20496\2408327102.py:61: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 0.1),
C:\Users\prags\AppData\Local\Temp\ipykernel_20496\2408327102.py:65: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float i

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-01-19 01:11:28,628] Trial 2 finished with value: 0.9973333333333333 and parameters: {'learning_rate': 0.002282714801166726, 'num_leaves': 90, 'max_depth': 21, 'min_data_in_leaf': 88, 'feature_fraction': 0.6842649836370481, 'bagging_fraction': 0.981895995597966, 'bagging_freq': 6, 'lambda_l1': 0.00031409195536767864, 'lambda_l2': 0.1345051728499768}. Best is trial 1 with value: 0.9974666666666667.
C:\Users\prags\AppData\Local\Temp\ipykernel_20496\2408327102.py:61: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 0.1),
C:\Users\prags\AppData\Local\Temp\ipykernel_20496\2408327102.py:65: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float 

...

[I 2025-01-19 01:11:37,913] Trial 48 finished with value: 1.0 and parameters: {'learning_rate': 0.07118427740086097, 'num_leaves': 182, 'max_depth': 13, 'min_data_in_leaf': 45, 'feature_fraction': 0.6263849828993812, 'bagging_fraction': 0.8224564828145631, 'bagging_freq': 4, 'lambda_l1': 0.16437738264385196, 'lambda_l2': 0.01545133090280987}. Best is trial 11 with value: 1.0.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

C:\Users\prags\AppData\Local\Temp\ipykernel_20496\2408327102.py:61: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 0.1),
C:\Users\prags\AppData\Local\Temp\ipykernel_20496\2408327102.py:65: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 1.0),
C:\Users\prags\AppData\Local\Temp\ipykernel_20496\2408327102.py:66: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.6, 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

AssertionError: The shape of the shap_values matrix does not match the shape of the provided data matrix.